In [1]:
import torchaudio
from torchaudio.transforms import Resample
from denoiser import pretrained
from IPython.display import Audio
import torch
import whisperx

/home/harry/ytsum/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/harry/ytsum/.venv/lib/python3.10/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/harry/ytsum/.venv/lib/python3.10/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures


In [3]:
denoiser =  pretrained.dns64().cpu().eval()

@torch.inference_mode()
def denoise(audio_file, sr=None):
    global denoiser
    if type(audio_file) is str: 
        audio, sr = torchaudio.load(audio_file)
    else:
        audio = audio_file
        assert sr is not None, "You must provide sample rate for loaded audio"
    
    audio = audio.sum(dim=0, keepdim=True)
    audio = Resample(sr, denoiser.sample_rate)(audio)
    B = 40
    denoiser = denoiser.cuda()
    wav = audio.split(B*denoiser.sample_rate, dim=1)
    denoised = []
    for w in wav:
        denoised.append(denoiser(w.cuda()))
    denoiser = denoiser.cpu()
    denoised = torch.cat(denoised, dim=-1)
    return denoised

In [4]:
abs = denoise("/home/harry/abs.opus")

In [5]:
from denoiser import pretrained
denoiser =  pretrained.dns64().cpu().eval()

device = "cuda" 
batch_size = 16
compute_type = "float16"
whisper = whisperx.load_model("large-v3", "cuda", compute_type=compute_type)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.2+cu121. Bad things might happen unless you revert torch to 1.x.


In [6]:
@torch.inference_mode()
def transcribe(audio):
    global whisper
    result = whisper.transcribe(audio.cpu().numpy(), batch_size=batch_size)
    alltext = ""

    for s in result["segments"]:
        alltext += s["text"]
        
    print("number of words", len(alltext.split(" ")))
    return result

results = transcribe(abs.flatten())

Detected language: en (0.99) in first 30s of audio...
number of words 4074


In [15]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
                                           
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16, attn_implementation="flash_attention_2")

# text = "Hello my name is"
# + inputs = tokenizer(text, return_tensors="pt").to(0)

# outputs = model.generate(**inputs, max_new_tokens=20)


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:31<00:00,  1.63s/it]


In [13]:
pre_prompt="""
You will receive chunks of a transcript. You won't know who is saying what but should summarise the point that are made.

Points:

Brian Johnson is spending $2m a year to stay young
Using algorithmic precision
In peak performance of entire life
Extended my lifespan over 30%
Reduced age by 12 years
Increased muscle and strength
Six months of perfect sleep

"""

prompt="""

Transcript Title: Abraham Voice Note

Transcript Content:

{transcript}

"""


final_prompt="""
You've produced this list of points from a podcast transcript:

Points:

{points}

Remove duplicated and produce a paragraph summary of the points.
"""

In [9]:
def generate(pre_prompt:str, prompt):
    messages = [
        {"role": "user", "content": pre_prompt},
        {"role": "assistant", "content": "Ok great I'll start extracting points from content using this template"},
        {"role": "user", "content": prompt},
    ]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    # model_inputs = tokenize(encodeds, return_tensors="pt").to(device)
    generated_ids = model.generate(encodeds, max_new_tokens=200, do_sample=True)
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [11]:
import re

segments = [s["text"] for s in results["segments"]]
all_text = "\n".join(segments)
result = re.split(r'[.?!…]+', all_text)
idxs = [m.start() for m in re.finditer(r'[.!?]', all_text)]
pidx = [0] + [idx + 1 for idx in idxs]  # +1 to start the next sentence after the punctuation
sentences_per_chunk = 10

chunks = []
for i in range(0, len(pidx), sentences_per_chunk):
    chunks.append("".join(all_text[pidx[j]:pidx[j+1]] for j in range(i, min(i+sentences_per_chunk, len(pidx)-1))))

In [14]:
points = []
for c in chunks:
    out = generate(pre_prompt, prompt.format(transcript=c))
    print(c)
    print(out[out.rfind("Points:"):])
    print("\n-----\n")
    points += out[out.rfind("Points:"):].replace("\n", "").split("*")[1:]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Hey Ari, just listening to the update. Apologies I wasn't here yesterday, but I really appreciate it, man. I appreciate you pushing forward with it as well. So completely followed what you were saying there. That was really nice, actually. Thank you so much for the... It was actually really nice and succinct. I completely understood where the problem was coming from.
Points:

* Abraham wasn't present but listened to the update later
* Appreciates Ari for pushing forward
* Understood the problem Ari was discussing
* Thankful for the update and found it succinct.

-----



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 So thank you for explaining the problem space that you're trying to deal with there.
 And the idea that you can generate the TTS, like TTS has gone so good, especially with the stuff that you're working on, that you're able to sort of generate it and then use it as the data sets themselves and then avoid the labeling and learning problem quite easily because you're the one that's generating the data and then getting it to self-label. As far as I understand, right, that sounds like it would be a really, really good way to be able to
 uh to get a much much more thoroughly trained model out there and then apply it to real world data and see how it handles it because honestly from the stuff that you send me like
 If you're able to generate that with, I don't know, let's say even 100 voices, 100 different voices, 100 different speech patterns with 100 different intonations and 100 different scenarios, like you were saying, like football commentary and stuff like that, all the different wil

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 I'm telling you now, most of them have no idea how to access that yet.
 Right? We're still a year to two years away from that even being remotely viable for most people. Most people are just content creators. They just wanted content create. So if you can wrap it in a content creation solution, and I'm not saying like, you know, add all the bells and whistles to it, but just something simple that is just literally in an app format that literally just creates realistic AI voices for a cheap price or whatever. Like, oh yeah, hell yeah. That sounds amazing. And then you just tweak it a bit more to be kind of like content creatively focusing. Like, hey,
 can immediately import this on top of that or whatever or hey you can you can get ai to train on your voice like you know like let's say you're a youtube content creator that's got hundreds of videos on his channel and he's just sick and he just wants to pump out videos faster so you have ai train on their on their voice sets like that's 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 And the second thing is like voiceovers and doing everything else that they need to do, right? Like the voiceovers on these videos and everything else that they want to do with them. So like, why not be able to just focus on that?
 why why not why not be able to just take basically what's already there and then just optimize it because you have a you have a good version of it you're able to then stick it into a like wrap it up in a nice and neat tidier package that's just so much more convenient for these users um or hyper tailored to these users because that's what it is discord it's just a hyper tailored solution
 It's just another chat forum. It's like Slack or whatever, but it was just hyper-focused for gamers. And so it took over the market. The moment that something is hyper-focused for a group of people, it's also hyper-convenient for that group of people. It solves all of their problems for those sets of people.
 Like the Opera GX browser, the only way that the Opera GX browse

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Why? Because it's got everything that a gamer needs built right into it. It's because you can compete.
 Anything extra that other browsers can't? No, technology really can't. But it thrives in the gaming community because it's hyper-focused for that shit. Visual Studio Code is just hyper-focused for the new dev experience. It's nice that CleanA works better than whatever the fuck we had before, Sublime Text 2 or the Atom, even the Atom code creator wasn't that bad, right? But like,
 And that's why this new age is all about hyper-tailoring these solutions, which is actually very, very easy to do because you're not reinventing the wheel any given time. The moment that you don't reinvent the wheel, you're able to pump out these ideas faster.
Points:

1. Browser specifically built for gaming needs
2. Compete better with other browsers
3. Hyper-focused for gaming community
4. Visual Studio Code is also hyper-focused for new dev experience
5. CleanA works better than previous versions like 